In [2]:
%pylab widget
%load_ext autoreload
%autoreload 2

from analysis_functions import *
import fastcat.fastcat as fc

from scipy.optimize import minimize

Populating the interactive namespace from numpy and matplotlib
The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


# First lets look at the GOS

In [3]:
nviews = 360

s = fc.Spectrum()
s.load('Varian_truebeam_phasespace')
kernel = fc.Kernel(s,'CuGOS-784-micrometer')
angles = np.linspace(0,np.pi*2,nviews)        

phantom = fc.Catphan_404()
phantom.phan_map = ['air','CATPHAN_Polystyrene','CATPHAN_Acrylic','CATPHAN_B20','CATPHAN_Delrin_revised','water','CATPHAN_Teflon_revised','air','CATPHAN_PMP','CATPHAN_B50','CATPHAN_LDPE','water','CATPHAN_Polystyrene','air','CATPHAN_Acrylic'] 
phantom.return_projs(kernel,s,angles,det_on=True,scat_on=False,mgy=7000/nviews)

phantom.reconstruct('FDK',filt='ram_lak')

(19, 41, 41) 19
really doing it!


ValueError: operands could not be broadcast together with shapes (360,64,512) (360,10,512) (360,64,512) 

In [3]:
plt.figure()
plt.imshow(np.rot90(phantom.img[4]))

recon_slice = phantom.img[5]
im = create_mask(recon_slice.shape)
contrast = []

for ii in range(2,int(np.max(im)+1)):
    
    contrast.append(np.mean(recon_slice[im == ii]))

# Get rid of the delrin
cc = np.array(sorted(contrast))[np.arange(len(contrast))]
star_data = np.array(star_data_gos[1::2])[np.arange(len(contrast))]

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [4]:
def HU_fun(a,x,y):
    return np.mean(np.abs((x - a[0])/(a[1] - a[0])*-1000 - y))

res = minimize(HU_fun,[cc[3],cc[0]],args=(cc,star_data))

x = [0,0,.853,0.945,.998,1.147,1.36,1.87]
plt.figure()
plt.plot(x,(cc - res.x[0])/(res.x[1] - res.x[0])*-1000,'s',c='cornflowerblue',label='fastCAT')
plt.plot(x,star_data,'s',c='darkorange',label='Star-lack')

fc_data = (cc - res.x[0])/(res.x[1] - res.x[0])*-1000

xx = np.linspace(0,max(x),100)
pp = np.polyfit(x,fc_data,1,full=1)
qq = np.polyfit(x,star_data,1,full=1)
yy = np.polyval(pp[0],xx)
zz = np.polyval(qq[0],xx)

plt.ylabel('HU')
plt.xlabel('Relative electron density')
plt.title('CWO')
plt.xticks(x,['air1','air2','PMP','LDPE','Poly','Acrylic','Delrin','Teflon'],rotation=75)
plt.plot(xx,yy,':',c='cornflowerblue',lw=0.8)
plt.plot(xx,zz,'-.',c='darkorange',lw = 0.8)
plt.legend()
plt.tight_layout()
plt.savefig('initial_cwo')

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [5]:
1-pp[1]/np.std(fc_data)**2

array([0.9995497])

In [6]:
1-qq[1]/np.std(star_data)**2

array([0.99137546])

# Now lets look at the CWO

In [7]:
nviews = 360

s = fc.Spectrum()
s.load('Varian_truebeam_phasespace')
kernel = fc.Kernel(s,'CWO-784-micrometer')
angles = np.linspace(0,np.pi*2,nviews)        

phantom = fc.Catphan_404()
phantom.phan_map = ['air','CATPHAN_Polystyrene','CATPHAN_Acrylic','CATPHAN_B20','CATPHAN_Delrin_revised','water','CATPHAN_Teflon_revised','air','CATPHAN_PMP','CATPHAN_B50','CATPHAN_LDPE','water','CATPHAN_Polystyrene','air','CATPHAN_Acrylic'] 
phantom.return_projs(kernel,s,angles,det_on=True,scat_on=False,mgy=700/nviews)

(17, 25, 25) 17
really doing it!
ratio is 2724.989798289916 number of photons 0.4242537489047066


In [8]:
phantom.reconstruct('FDK',filt='ram_lak')

In [9]:
plt.figure()
plt.imshow(np.rot90(phantom.img[4]))

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [10]:
recon_slice = phantom.img[5]
im = create_mask(recon_slice.shape)
contrast = []

for ii in range(2,int(np.max(im)+1)):
    
    contrast.append(np.mean(recon_slice[im == ii]))

In [11]:
star_data_cwo = [0.0041972717733472775, -864.2384105960264,
0.0041972717733472775, -880.7947019867552,
0.8541448058761805, -105.96026490066242,
0.9443861490031478, -3.3112582781459423,
1.0031479538300103, 39.73509933774835,
1.1521511017838406, 152.31788079470198,
1.359916054564533, 364.23841059602637,
1.869884575026233, 837.7483443708609]

# Get rid of the delrin
cc = np.array(sorted(contrast))
star_data = np.array(star_data_cwo[1::2])

In [12]:
def HU_fun(a,x,y):
    return np.mean(np.abs((x - a[0])/(a[1] - a[0])*-1000 - y))

res = minimize(HU_fun,[cc[3],cc[0]],args=(cc,star_data))

x = [0,0,.853,0.945,.998,1.147,1.36,1.87]
plt.figure()
plt.plot(x,(cc - res.x[0])/(res.x[1] - res.x[0])*-1000,'s',c='cornflowerblue',label='fastCAT')
plt.plot(x,star_data,'s',c='darkorange',label='Star-lack')

fc_data = (cc - res.x[0])/(res.x[1] - res.x[0])*-1000

xx = np.linspace(0,max(x),100)
pp = np.polyfit(x,fc_data,1,full=1)
qq = np.polyfit(x,star_data,1,full=1)
yy = np.polyval(pp[0],xx)
zz = np.polyval(qq[0],xx)

plt.ylabel('HU')
plt.xlabel('Relative electron density')
plt.title('CWO')
plt.xticks(x,['air','air','PMP','LDPE','Poly','Acrylic','Delrin','Teflon'],rotation=75)
plt.plot(xx,yy,':',c='cornflowerblue',lw=0.8)
plt.plot(xx,zz,'-.',c='darkorange',lw = 0.8)
plt.legend()
plt.tight_layout()
plt.savefig('initial_cwo')

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [51]:
1-pp[1]/np.std(fc_data)**2

array([0.99967508])

In [52]:
1-qq[1]/np.std(star_data)**2

array([0.99746053])

# Now lets look at the CsI numbers

In [2]:
nviews = 360

s = fc.calculate_spectrum(100, 12, 3, 50)
s.attenuate(0.2,fc.get_mu(z=13))
kernel = fc.Kernel(s,'CsI-784-micrometer')
angles = np.linspace(0,np.pi*2,nviews)        

phantom = fc.Catphan_404()
phantom.phan_map = ['air','CATPHAN_Polystyrene','CATPHAN_Acrylic','CATPHAN_B20','CATPHAN_Delrin_revised','water','CATPHAN_Teflon_revised','air','CATPHAN_PMP','CATPHAN_B50','CATPHAN_LDPE','water','CATPHAN_Polystyrene','air','CATPHAN_Acrylic'] 
phantom.return_projs(kernel,s,angles,det_on=True,scat_on=False,mgy=7000/nviews)

phantom.reconstruct('FDK',filt='ram_lak')

Calculation:  1 / 50
Calculation:  2 / 50
Calculation:  3 / 50
Calculation:  4 / 50
Calculation:  5 / 50
Calculation:  6 / 50
Calculation:  7 / 50
Calculation:  8 / 50
Calculation:  9 / 50
Calculation:  10 / 50
Calculation:  11 / 50
Calculation:  12 / 50
Calculation:  13 / 50
Calculation:  14 / 50
Calculation:  15 / 50
Calculation:  16 / 50
Calculation:  17 / 50
Calculation:  18 / 50
Calculation:  19 / 50
Calculation:  20 / 50
Calculation:  21 / 50
Calculation:  22 / 50
Calculation:  23 / 50
Calculation:  24 / 50
Calculation:  25 / 50
Calculation:  26 / 50
Calculation:  27 / 50
Calculation:  28 / 50
Calculation:  29 / 50
Calculation:  30 / 50
Calculation:  31 / 50
Calculation:  32 / 50
Calculation:  33 / 50
Calculation:  34 / 50
Calculation:  35 / 50
Calculation:  36 / 50
Calculation:  37 / 50
Calculation:  38 / 50
Calculation:  39 / 50
Calculation:  40 / 50
Calculation:  41 / 50
Calculation:  42 / 50
Calculation:  43 / 50
Calculation:  44 / 50
Calculation:  45 / 50
Calculation:  46 / 

In [3]:
plt.figure()
plt.imshow(np.rot90(phantom.img[4]))

recon_slice = phantom.img[5]
im = create_mask(recon_slice.shape)
contrast = []

for ii in range(2,int(np.max(im)+1)):
    
    contrast.append(np.mean(recon_slice[im == ii]))

# Get rid of the delrin
cc = np.array(sorted(contrast))[np.arange(len(contrast))]
star_data = np.array(star_data_gos[1::2])[np.arange(len(contrast))]

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [4]:
def HU_fun(a,x,y):
    return np.mean(np.abs((x - a[0])/(a[1] - a[0])*-1000 - y))

res = minimize(HU_fun,[cc[3],cc[0]],args=(cc,star_data))

x = [0,0,.853,0.945,.998,1.147,1.36,1.87]
plt.figure()
plt.plot(x,(cc - res.x[0])/(res.x[1] - res.x[0])*-1000,'s',c='cornflowerblue',label='fastCAT')
plt.plot(x,star_data,'s',c='darkorange',label='Star-lack')

fc_data = (cc - res.x[0])/(res.x[1] - res.x[0])*-1000

xx = np.linspace(0,max(x),100)
pp = np.polyfit(x,fc_data,1,full=1)
qq = np.polyfit(x,star_data,1,full=1)
yy = np.polyval(pp[0],xx)
zz = np.polyval(qq[0],xx)

plt.ylabel('HU')
plt.xlabel('Relative electron density')
plt.title('CWO')
plt.xticks(x,['air1','air2','PMP','LDPE','Poly','Acrylic','Delrin','Teflon'],rotation=75)
plt.plot(xx,yy,':',c='cornflowerblue',lw=0.8)
plt.plot(xx,zz,'-.',c='darkorange',lw = 0.8)
plt.legend()
plt.tight_layout()
plt.savefig('initial_cwo')

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [5]:
1-pp[1]/np.std(fc_data)**2

array([0.9995497])

In [6]:
1-qq[1]/np.std(star_data)**2

array([0.99137546])